# Simple Rating
* Predicts the global mean across all users and items

In [ ]:
medium = "";

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../TrainingAlpha.ipynb");

In [ ]:
import ProgressMeter: @showprogress
import Statistics: mean
import StatsBase

In [ ]:
const name = "$medium/Simple/drop";

In [ ]:
function train_model()
    df = as_metric(get_split("training", "train", medium, [:itemid, :status]), "drop")
    counts = StatsBase.countmap(df.itemid)
    drops = StatsBase.countmap(df.itemid[df.metric.==1])
    drop_rate = Dict{Int32,Float32}()
    for k in keys(counts)
        drop_rate[k] = get(drops, k, 0) / counts[k]
    end
    a = Float64[get(drop_rate, i, 0) for i = 1:num_items(medium)]
    write_params(Dict("a" => convert.(Float32, a)), name)
end;

In [ ]:
function model(rd::RatingsDataset, userids::Set{Int32})
    params = read_params(name)
    a = params["a"]
    preds = Dict{Int32,Vector{Float32}}()
    @showprogress for u in userids
        preds[u] = copy(a)
    end
    preds
end;

In [ ]:
train_model();

In [ ]:
write_alpha(model, medium, name)
print_losses(medium, "drop", [name]);